<a href="https://colab.research.google.com/github/profesorKiki/EjemplosPython/blob/main/Ejemplos/Procesos/Ejemplos_De_Creaci%C3%B3n_De_Procesos_NOHUP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica de procesos

Ejemplo en **Python**

In [1]:
%%writefile fork.py
import sys
import os
import time

MAX_CHILD = 2

print( f"Proceso padre ({ os.getpid() })", flush=True )

for i in range(MAX_CHILD):
    pid = os.fork()

    if pid < 0:
        sys.exit("Error al crear el nuevo proceso")

    if pid == 0:
        print(f"Hijo {i+1} ({ os.getpid() }) - Mi padre es ({ os.getppid() })",flush=True)
        time.sleep(15)
        os._exit(0)

for _ in range(2):
    print( os.wait() )


Writing fork.py


In [61]:
!nohup python fork.py 1>salida 2>/dev/null &

In [62]:
!ps -aef


UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 21:17 ?        00:00:00 /sbin/docker-init -- /datala
root           7       1  0 21:17 ?        00:00:02 /tools/node/bin/node /datala
root          15       7  0 21:17 ?        00:00:00 /bin/bash -e /usr/local/cola
root          17       1  0 21:17 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          19      17  0 21:17 ?        00:00:00 /usr/colab/bin/kernel_manage
root          41       0  0 21:17 ?        00:00:00 tail -n +0 -F /root/.config/
root          47       0  0 21:17 ?        00:00:00 tail -n +0 -F /root/.config/
root          65       7  1 21:17 ?        00:00:19 [python3] <defunct>
root          66       7  0 21:17 ?        00:00:00 python3 /usr/local/bin/colab
root          87       7  0 21:17 ?        00:00:06 /usr/bin/python3 /usr/local/
root          88       7  0 21:17 ?        00:00:00 /usr/local/bin/dap_multiplex
root         389      87  0 21:18 ?        00:00:06 /usr/bin/p

In [63]:
!pgrep -f "fork.py" | head -n 1

4446


In [23]:
!pstree -pc $(pidof "python fork.py")

docker-init(1)─┬─node(7)─┬─colab-fileshim.(66)
               │         ├─dap_multiplexer(88)─┬─{dap_multiplexer}(89)
               │         │                     ├─{dap_multiplexer}(90)
               │         │                     └─{dap_multiplexer}(91)
               │         ├─jupyter-noteboo(87)─┬─python3(389)─┬─pstree(1914)
               │         │                     │              ├─{python3}(393)
               │         │                     │              ├─{python3}(394)
               │         │                     │              ├─{python3}(395)
               │         │                     │              ├─{python3}(396)
               │         │                     │              ├─{python3}(397)
               │         │                     │              ├─{python3}(398)
               │         │                     │              ├─{python3}(399)
               │         │                     │              ├─{python3}(400)
               │         │     

Ejemplo en **Java**

In [ ]:
%%writefile Padre.java
import java.io.IOException;

public class Padre
{
    public static void main(String[] args) throws IOException,  InterruptedException
    {
        ProcessBuilder pb = new ProcessBuilder("java", "Hijo.java");
        pb.redirectErrorStream(true);
        pb.inheritIO();

        Process proceso = pb.start();

        int error = proceso.waitFor();

        if( error != 0 )
        {
            System.out.println("Error al ejecutar el programa");
        }
    }
}

Overwriting Padre.java


In [ ]:
%%writefile Hijo.java
class Hijo
{
    public static void main(String[] args) throws InterruptedException
    {
        ProcessHandle yo = ProcessHandle.current();
        long pid  = yo.pid();
        long ppid = yo.parent().get().pid();
        System.out.println("Hijo: " + pid + ". Padre: " + ppid );
        Thread.sleep(15000);
    }
}

Overwriting Hijo.java


In [ ]:
!nohup java Padre.java 1>salida 2>/dev/null &

In [ ]:
!pstree -pT 27508

java(27508)───java(27531)
